In [1]:
import pickle

with open('all_ids.pickle', 'rb') as f:
    ids = pickle.load(f)

In [2]:
import numpy as np

from doublegis_api.api import Api2Gis

api = Api2Gis()
api.load()
api.describe()

Регион: Санкт-Петербург
Количество рубрик: 27
Количество смежных рубрик: 1326
Количество станций метро: 66
Количество зданий: 52543
Количество организаций: 104186
Количество филиалов: 156165


In [3]:
api.add_metro_distances()

In [4]:
api.get_removed_orgs_by_ids(ids=ids, progress='console', verbose=True)

До обновления имеется 156165 филиалов
До обновления имеется 1845 удалённых филиалов


 11%|█         | 345/3236 [09:09<42:08,  1.14it/s]t]

TypeError: 'NoneType' object is not iterable

 33%|███▎      | 1071/3236 [15:17<15:08,  2.38it/s] 